In [1]:
from housingresearch.systems import CensusClient
from housingresearch.config import settings
from us import states

Initialize a `CensusClient` explorer.

In [2]:
c = CensusClient()

Get a variable across a set of census tracts

In [ ]:
old_homes = c.census.acs5.state_place_tract(
    fields=("NAME", "B25034_010E"),
    state=states.FL.fips,
    place=63000,
    year=2019,
    return_geometry=False,
)

old_homes[0:2]

Get median rent for each census tract in Saint Petersburg. 

- B25031_001E is median rent for a studio
- B25031_002E is median rent for a one-bedroom
- B25031_003E is median rent for a two-bedroom

In [ ]:
median_rent = c.census.acs5.state_place_tract(
    fields=("NAME", "B25031_001E", "B25031_002E", "B25031_003E"),
    state=states.FL.fips,
    place=63000,
    year=2019,
    return_geometry=False,
)

median_rent[0:2]